In [1]:
api_key = ""
channel_id = ""

In [2]:
from googleapiclient.discovery import build
youtube = build("youtube", "v3", developerKey=api_key)

In [3]:
def get_channel_videos(channel_id):
    res = youtube.channels().list(id=channel_id,
                                 part="contentDetails").execute()
    playlist_id = res["items"][0]["contentDetails"]["relatedPlaylists"]["uploads"]
    
    videos = []
    next_page_token = None
    
    while 1:
        res = youtube.playlistItems().list(playlistId=playlist_id,
                                          part="snippet",
                                          maxResults=50,
                                          pageToken=next_page_token).execute()
        videos += res["items"]
        next_page_token = res.get("nextPageToken")
        
        if next_page_token is None:
            break
    
    return videos

In [4]:
videos = get_channel_videos(channel_id)

In [5]:
res = youtube.videos().list(id=videos[0]["snippet"]["resourceId"]["videoId"],
                           part="statistics").execute()

In [6]:
video_ids = list(map(lambda x:x['snippet']['resourceId']['videoId'], videos))

In [7]:
def get_videos_stats(video_ids):
    stats = []
    for i in range(0, len(video_ids), 50):
        res = youtube.videos().list(id=','.join(video_ids[i:i+50]),
                                   part='statistics').execute()
        stats += res['items']
    return stats

In [8]:
stats = get_videos_stats(video_ids)

In [65]:
publishedAt = []
title = []
viewCount = []
for i in range(len(videos)):
    publishedAt.append(videos[i]['snippet']['publishedAt'])
    title.append(videos[i]['snippet']['title'])
    viewCount.append(stats[i]['statistics']['viewCount'])

In [66]:
dict = {}
dict["publishedAt"] = publishedAt
dict["title"] = title
dict["viewCount"] = viewCount

In [67]:
import pandas as pd
df = pd.DataFrame.from_dict(dict)

In [104]:
df['viewCount'] = df['viewCount'].astype(int)

In [105]:
type(df['viewCount'][0])

numpy.int32

In [106]:
df.head()

,publishedAt,title,viewCount
0,2021-04-08T08:00:09Z,에이치엘비 CAR-T 상업화 최소 3년이상 앞당길 수 있다!! CAR-T세포가 활성...,1928
1,2021-04-08T01:34:11Z,에이치엘비 주주 오세훈 서울시장 당선!!,3779
2,2021-04-07T21:00:15Z,에이치엘비 진양곤 회장은 코르키로부터 확보했던 넥스트사이언스의 메자닌에 대한 콜옵션...,6545
3,2021-04-07T08:15:01Z,에이치엘비 증선위 뉴스기사가 없는 것으로 보아 열리지 않고 무기한 연기 가능성!!,7487
4,2021-04-07T03:00:11Z,에이치엘비가 뛰어든 CAR-T치료제의 부작용은 노바티스와 길리어드사이언스에서도 보이...,3160


# 에이치엘비를 포함하지 않는 제목의 특징

In [107]:
df_copy = df.copy()

In [108]:
# https://ponyozzang.tistory.com/622
df_copy['hlb'] = df_copy['title'].str.contains('에이치엘비')

In [109]:
df_copy.tail()

,publishedAt,title,viewCount,hlb
2586,2018-10-05T02:22:45Z,빅토리의5번째주식단타매매영상,47,False
2587,2018-10-04T15:50:27Z,빅토리의4번째주식단타매매영상,53,False
2588,2018-10-04T11:02:07Z,빅토리의3번째주식단타매매영상,18,False
2589,2018-10-04T10:45:30Z,빅토리의2번째주식단타매매영상,97,False
2590,2018-10-04T10:04:42Z,빅토리의1번째주식단타매매영상,277,False


In [110]:
df_copy.groupby('hlb').viewCount.mean()

hlb
False     592.218150
True     5859.741869
Name: viewCount, dtype: float64

In [111]:
df_copy.groupby('hlb').viewCount.std()

hlb
False    1165.081253
True     4550.089526
Name: viewCount, dtype: float64

In [120]:
# https://stackoverflow.com/questions/17071871/how-to-select-rows-from-a-dataframe-based-on-column-values
df_nothlb = df_copy.loc[(df_copy['hlb'] == False)]

In [121]:
df_nothlb.head()

,publishedAt,title,viewCount,hlb
27,2021-03-31T09:01:52Z,넥스트사이언스 매매의 정석: 우연히 저점매수 고점매도를 한 사람이 있다?!,4307,False
31,2021-03-30T07:58:58Z,넥스트사이언스 감사보고서 적정공시 후 시원한 슈팅급등!! 나노젠의 코로나치료제를 부...,4067,False
76,2021-03-17T08:46:19Z,"넥스트사이언스 아임얼라이브 콤부차 국제 단체 KBI 등록, 프리바이오틱스 풍부",4335,False
226,2021-02-11T06:44:13Z,메지온 P-Value가 변동된 이유는 무엇일까?,841,False
229,2021-02-09T04:09:56Z,옵티머스 뒤에 모피아가 있었다?! 하나은행은 왜 느슨했을까,1935,False


In [122]:
len(df_nothlb)

1146

# 조회수 많은 순으로 정렬한다.

In [136]:
df_nothlb = df_nothlb.sort_values(by='viewCount', ascending=False)

# show all

In [137]:
# https://dev.to/chanduthedev/how-to-display-all-rows-from-data-frame-using-pandas-dha
pd.set_option('display.max_rows', 1200)

In [138]:
df_nothlb[:5]

,publishedAt,title,viewCount,hlb
1192,2020-03-05T04:54:29Z,대규모 유상증자를 하면 주가는 어떻게 되나요?,12896,False
1831,2019-08-18T10:41:53Z,"단일가매매 이후 주가는 어떻게 되는가? | 단기과열종목(램테크놀러지, 일지테크, 미...",11024,False
1450,2019-12-21T15:52:21Z,26일 신규상장하는 천랩의 2대주주가 라이프리버라고?!,10168,False
342,2020-12-31T07:07:59Z,박셀바이오 지금까지 제가 생각하고 느껴왔던 모든 것을 말씀드립니다. 새해복 많이 받...,10031,False
1569,2019-10-26T02:54:00Z,키움실투대회 돌파매매는 손절이 중요하다. 아무리봐도 마하세븐님같은데 2019-10-...,9533,False


# to_csv
참고 링크 : https://dolhani.tistory.com/521  
dataframe을 엑셀로 저장하면 '한글 깨짐' 현상이 나타난다. 'utf-8'이 아닌 'utf-8-sig'로 인코딩하면 해결된다.

In [139]:
df_nothlb.to_csv('nothlb.csv', encoding='utf-8-sig')

# Results
  손익분기점 수익을 넘기는 조회수를 비교해보았다. 분석자료는 해당 엑셀파일에 첨부되어 있다. 조회수가 최소 8천회가 넘어야 손익분기점을 넘긴다.

# Conclusion
새로운 영상을 제작할 경우 조회수가 8천회를 넘는지를 기준으로 의사결정한다.